In [14]:
model_path="craft_mlt_25k-opset-11.onnx"
# model_path="E:/Projects/DATA/weights/detectors/EAST.pb"
# image_path = "E:/Projects/DATA/license_plate/plates/6.jpg"
image_path="D:/Projects/CRAFT-pytorch/data/1.jpg"
inpWidth = 768
inpHeight = 768
outNames = []

In [15]:
import cv2 as cv

In [16]:
frame = cv.imread(image_path)
# cv.imwrite('this-test.jpg',cv.imread())

In [17]:
height_ = frame.shape[0]
width_ = frame.shape[1]
rW = width_ / float(inpWidth)
rH = height_ / float(inpHeight)

AttributeError: 'NoneType' object has no attribute 'shape'

In [18]:
blob = cv.dnn.blobFromImage(frame, 1.0, (inpWidth, inpHeight), (123.68, 116.78, 103.94), True, False)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [19]:
detector = cv.dnn.readNet(model_path)

In [20]:
detector.setInput(blob)

In [21]:
outs = detector.forward(outNames)

In [22]:
outs[0]

array([[[[ 0.37910014,  3.1550484 ,  3.1556559 , ...,  0.        ,
           0.        ,  0.91519976],
         [ 2.6427748 ,  4.7952776 ,  4.754899  , ...,  0.2645709 ,
           0.6447325 ,  1.5957768 ],
         [ 2.1782484 ,  5.5275655 ,  5.6013675 , ...,  0.        ,
           0.29601383,  2.0807452 ],
         ...,
         [ 0.        ,  0.        ,  0.15090841, ...,  1.4222038 ,
           1.9904819 ,  4.92943   ],
         [ 0.        ,  2.1051016 ,  3.8372793 , ...,  5.6209497 ,
           5.2761908 ,  6.171085  ],
         [ 0.        ,  0.        ,  0.        , ...,  0.756297  ,
           0.5398922 ,  2.5956328 ]],

        [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        

array([[[0.0000000e+00, 9.5639199e-01, 1.2300520e+00, ...,
         8.7520725e-01, 0.0000000e+00, 1.6697695e+00],
        [2.0128304e-01, 6.3440382e-01, 8.0722207e-01, ...,
         3.6841157e-01, 1.3142371e-01, 4.3809247e-01],
        [0.0000000e+00, 2.1047781e-01, 2.2946441e-01, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         1.7738044e+00, 2.7602856e+00, 7.2756658e+00],
        [0.0000000e+00, 1.8147933e+00, 3.8001332e+00, ...,
         6.8967876e+00, 6.6519585e+00, 8.6941624e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         1.2702076e+00, 1.2142417e+00, 4.2170491e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 3.4533525e-01],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.000